In [1]:
# Loading my OPENAI KEY as an environment variable
from dotenv import load_dotenv
load_dotenv()

True

## Audio transcription

We begin by  transcribing the audio from the input

In [5]:
! git --version

git version 2.41.0.windows.1


In [3]:
import os

In [2]:
import whisper

audio_input_path = "C:/Users/anand/OneDrive/Desktop/pybin_2/RentOK Task Tutorial Generation/audio/audio_to_transcribe.mp3" 

model = whisper.load_model("base.en")

In [3]:
print(os.getcwd())
result = model.transcribe(audio_input_path)
print(result["text"])

c:\Users\anand\OneDrive\Desktop\pybin_2\RentOK Task Tutorial Generation


FileNotFoundError: [WinError 2] The system cannot find the file specified

In [4]:
a = int(input(""))

ValueError: invalid literal for int() with base 10: ''

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain, TransformChain
from langchain.prompts import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

## Steps generation

In [ ]:
# Description of the app we are using

# USER INPUT
app_desc = """VITian is an application built for students of VIT to access their student data. 
We can use the app to find out examination allotments."""

# Description of the actions we are performing

# USER INPUT
actions_desc = """We begin by opening the application.
Then we refresh the app time by pressing the refresh icon.
Then we open the sidebar.
Then we tap on "Academics"
Then we tap on "Exam Schedule"
We can now see the exam schedule and allocation."""

# Prompt template to generate teh narrator text
topic_template = """You are to convert the list of steps performed in the video into an organized format and properly number them. 
The output must be a python list of strings where each string is a step. Be informative and do not make up things.

App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The steps being performed in the video are:
```
{steps}
```

Sample inputs: 
```
App details: Gmail is an application used to send email to other people across the world.
Tutorial Description: The tutorial shows how to view a sent email in the Gmail app.
The steps being performed in the video are:
We open the gmail application on our mobile device,
we then need to navigate to the Sent emails page by tapping on the sidebar and then tapping on the Sent tab.
Now we can select the specific email and finally, we can view the email that we have sent out.  
```

Output example:
```
['Open the Gmail app',
'Open the sidebar',
'Select Sent',
'Select the you want to view',
'View the email']
```
"""

In [ ]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are an expert at making tutorial videos and very good at defining tasks in simple terms.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc","tutorial_desc", "steps"]))

In [ ]:
# Chain to write the create the steps from the input text
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
steps_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='steps')

In [ ]:
# testing the steps + prompt engineering

In [ ]:
# messages = [
#     SystemMessage(
#         content="You are a narrator with experience in making tech tutorial videos."
#     ),
#     HumanMessage(
#         content="""You are to convert the list of steps performed in the video into an organized format and properly number them. 
# The output must be a python list of strings where each string is a step. Be informative and do not make up things.

# App details:
# ```
# VITian is an application built for students of VIT to access their student data. 
# We can use the app to find out examination allotments.
# ```

# Tutorial Description:
# ```
# This is a tutorial on how to use the VITian app to view the exam schedule.
# ```

# The steps being performed in the video are:
# ```
# ['Open the VITian app',
# 'Click on the refresh icon',
# 'Open the sidebar',
# 'Select Academics',
# 'Select Exam Schedule',
# 'View the exam schedule and allocation']
# ```

# Sample inputs: 
# ```
# App details: Gmail is an application used to send email to other people across the world.
# Tutorial Description: The tutorial shows how to view a sent email in the Gmail app.
# The steps being performed in the video are:
# We open the gmail application on our mobile device,
# we then need to navigate to the Sent emails page by tapping on the sidebar and then tapping on the Sent tab.
# Now we can select the specific email and finally, we can view the email that we have sent out.  
# ```

# Output example:
# ```
# ['Open the Gmail app',
# 'Open the sidebar',
# 'Select Sent',
# 'Select the you want to view',
# 'View the email']
# ```
# """
#     ),
# ]

In [ ]:
# op = chat(messages)

In [ ]:
# op.content

"Here is the organized format of the steps performed in the video:\n\n```python\n[\n'Step 1: Open the VITian app',\n'Step 2: Click on the refresh icon',\n'Step 3: Open the sidebar',\n'Step 4: Select Academics',\n'Step 5: Select Exam Schedule',\n'Step 6: View the exam schedule and allocation'\n]\n```"

## Narrator Script Generation

In [ ]:
# Description of the app we are using

# USER INPUT
app_desc = """VITian is an application built for students of VIT to access their student data. 
We can use the app to find out examination allotments."""

# Description of the actions we are performing

# USER INPUT
actions_desc = """We begin by opening the application.
Then we refresh the app time by pressing the refresh icon.
Then we open the sidebar.
Then we tap on "Academics"
Then we tap on "Exam Schedule"
We can now see the exam schedule and allocation."""

# Prompt template to generate teh narrator text
topic_template = """You are to narrate a tutorial video about an app. Be informative and do not make up things.
App details:
```
{app_desc}
```

Tutorial Description:
```
{tutorial_desc}
```

The steps being performed in the video are:
```
{steps}
```
"""

In [ ]:
chat = ChatOpenAI(temperature=0, model_name = "gpt-4")
system_message_prompt = SystemMessage(content="You are a narrator with experience in making tech tutorial videos.")
human_message_prompt = HumanMessagePromptTemplate(prompt=PromptTemplate(
                                                  template=topic_template,
                                                  input_variables=["app_desc", "tutorial_desc", "steps"]))

In [ ]:
# Chain to write the script for the narrator
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
script_chain = LLMChain(llm=chat, prompt=chat_prompt_template, output_key='script')

In [ ]:
# testing the scripts + prompt engineering

In [ ]:
# messages = [
#     SystemMessage(
#         content="You are a narrator with experience in making tech tutorial videos."
#     ),
#     HumanMessage(
#         content="""You are to narrate a tutorial video about an app. Be informative and do not make up things.
# App details:
# ```
# VITian is an application built for students of VIT to access their student data. 
# We can use the app to find out examination allotments.
# ```

# Tutorial Description:
# ```
# This is a tutorial on how to use the VITian app to view the exam schedule.
# ```

# The steps being performed in the video are:
# ```
# [\n'Step 1: Open the VITian app',\n'Step 2: Click on the refresh icon',\n'Step 3: Open the sidebar',\n'Step 4: Select Academics',\n'Step 5: Select Exam Schedule',\n'Step 6: View the exam schedule and allocation'\n]
# ```
# Generate a maximum of one to two sentences per step.
# """
#     ),
# ]

In [ ]:
# op2 = chat(messages)

In [ ]:
# op2.content

'[Background Music Starts]\n\n"Hello everyone, welcome to our tutorial on how to use the VITian app to view your exam schedule. Let\'s get started.\n\nStep 1: First, locate and open the VITian app on your device. This is your gateway to all your student data.\n\nStep 2: Once the app is open, you\'ll see a refresh icon at the top of the screen. Click on this to ensure you have the most up-to-date information.\n\nStep 3: Now, let\'s navigate to the sidebar. You can do this by swiping from the left edge of your screen or tapping on the three horizontal lines at the top left corner.\n\nStep 4: In the sidebar, you\'ll find an option labeled \'Academics\'. Go ahead and select this to proceed.\n\nStep 5: Under \'Academics\', you\'ll see \'Exam Schedule\'. Tap on this to view all upcoming exams.\n\nStep 6: And there you have it! You can now view your exam schedule and allocation. This will help you plan your study schedule effectively.\n\nThat\'s all for this tutorial. Stay tuned for more tips

# Generate the audio